# 0. Open Database

In [1]:
import requests
import pandas as pd
import sqlite3
import tqdm
import requests as rq
from bs4 import BeautifulSoup
from io import BytesIO
import re

import myAPI_keys as Key
import init_db

init_db.init_entire_database()
print("Database initilized! load data from API and insert into database...")


conn = sqlite3.connect("db/StockManageAPP.sqlite")

cur = conn.cursor()

Database initilized! load data from API and insert into database...


/Users/jiho/Desktop/DB_API_assignment/init_db.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[1] = str(row[1]).zfill(6)
/Users/jiho/Desktop/DB_API_assignment/init_db.py:55: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[1] = str(row[1]).zfill(6)


# 1. APP Loading

## 1.1.1. Update [StockInfo] Table

## 1.1.1. Defince Function : Get_Stock_Price 
- 한국증권거래소 API에 종목코드를 파라미터로 이용해, 종목명과 종가 가져옴.

In [2]:
def Get_Stock_Price(_code):
    endpoint = "http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo"
    url = f"{endpoint}?pageNo={1}&resultType=json&numOfRows={1}&serviceKey={Key.STOCK_INFO_ENCODED_KEY}&likeSrtnCd={_code}"
    response = (
        requests.get(url).json().get("response").get("body").get("items").get("item")[0]
    )
    return response

## 1.1.2. Add IndividualStockAcount.StockCode to [StockInfo Table(df)]
- individual stock info table에서 stock code들을 모두 stock info에 적어야함

In [3]:
# _tmp = cur.execute("SELECT DISTINCT StockCode FROM IndividualStockAcount")
df = pd.read_sql_query("SELECT DISTINCT StockCode FROM IndividualStockAcount", conn)
df["StockName"] = ""
df["Category"] = ""
df["Price"] = ""

In [4]:
df

,StockCode,StockName,Category,Price
0,000660,,,
1,005930,,,
2,035420,,,


## 1.1.2. load category table (in memory)
- http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020502
- KRX에서 업종정보 포함한 종합 정보 csv file 다운 후, 메모리에 불러옴

In [5]:
# Load the two CSV files with 'euc-kr' encoding
df1 = pd.read_csv("db/doc/data_2239_20231121.csv", encoding="euc-kr")
df2 = pd.read_csv("db/doc/data_2303_20231121.csv", encoding="euc-kr")

# Concatenate the two DataFrames
df3 = pd.concat([df1, df2])

# If you want to reset the index
krx_sector = df3.reset_index(drop=True)

## 1.1.3. Add Stock{Name, Price} to [StockInfo(df)] by using API

In [6]:
for _StockCode in tqdm.tqdm(df["StockCode"]):
    # Get the StockName and Price from the response

    _ans = Get_Stock_Price(_StockCode)
    _StockName = _ans["itmsNm"]
    _Price = _ans["clpr"]
    ###############################################################################
    # Find the matching row in krx_sector
    matching_sector = krx_sector[krx_sector["종목코드"] == _StockCode]["업종명"]

    df.loc[df["StockCode"] == _StockCode, "Category"] = matching_sector.values[0]
    ###############################################################################

    # print(_StockName, _Price)
    # Assign the value to the StockName column of the matching row
    df.loc[df["StockCode"] == _StockCode, "StockName"] = _StockName
    df.loc[df["StockCode"] == _StockCode, "Price"] = _Price


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:05<00:00,  1.91s/it]


In [7]:
df

,StockCode,StockName,Category,Price
0,000660,SK하이닉스,전기전자,129900
1,005930,삼성전자,전기전자,72500
2,035420,NAVER,서비스업,202000


## 1.1.4. 만든 df Table을 [StockInfo table]에 대입

In [8]:
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Create the INSERT statement
    sql = f"""
    INSERT INTO StockInfo (StockCode, StockName, Price, Category)
    VALUES ('{row['StockCode']}', '{row['StockName']}', {row['Price']}, '{row['Category']}')
    """

    # Execute the INSERT statement
    cur.execute(sql)

# Commit the changes and close the connection
conn.commit()

# 1.2. Update [UserInfo] Table 
- Update a [UserInfo].Blance
- Update a [UserInfo].FavoriteCategory

## 1.2.1 Update [UserInfo].Blance

In [9]:
sql = f"""
    SELECT ID, SUM(Count * Price) as TotalValue
    FROM (
        SELECT IndividualStockAcount.ID, IndividualStockAcount.StockCode, IndividualStockAcount.Count, StockInfo.Price
        FROM IndividualStockAcount
        JOIN StockInfo ON IndividualStockAcount.StockCode = StockInfo.StockCode
    )
    GROUP BY ID
    """

df = pd.read_sql_query(sql, conn)

In [10]:
df

,ID,TotalValue
0,jiho266,867000
1,jiho267,808000
2,jiho268,649500
3,jiho270,404400


In [11]:
# Write the DataFrame to a temporary table in the SQLite database
df.to_sql("temp", conn, if_exists="replace", index=False)

# Update the UserInfo table based on the temporary table
cur.execute(
    """
    UPDATE UserInfo
    SET Balance = (
        SELECT TotalValue
        FROM temp
        WHERE temp.ID = UserInfo.ID
    )
    WHERE EXISTS (
        SELECT 1
        FROM temp
        WHERE temp.ID = UserInfo.ID
    )
"""
)
cur.execute("DROP TABLE IF EXISTS temp")  

conn.commit()

## 1.2.2. [IndividualStockAccount].TotalPrice

In [12]:
sql = f"""
    SELECT IndividualStockAcount.ID, IndividualStockAcount.StockCode, IndividualStockAcount.Count, StockInfo.Price
    FROM IndividualStockAcount
    JOIN StockInfo ON IndividualStockAcount.StockCode = StockInfo.StockCode
    """

df = pd.read_sql_query(sql, conn)

In [13]:
df

,ID,StockCode,Count,Price
0,jiho266,005930,3,72500
1,jiho267,035420,4,202000
2,jiho268,000660,5,129900
3,jiho266,000660,5,129900
4,jiho270,000660,1,129900
5,jiho270,005930,1,72500
6,jiho270,035420,1,202000


In [14]:
# Write the DataFrame to a temporary table in the SQLite database
df.to_sql("temp", conn, if_exists="replace", index=False)

# Update the individualstockaccount table based on the temporary table
cur.execute(
    """
    UPDATE IndividualStockAcount
    SET TotalPrice = (
        SELECT Count * Price
        FROM temp
        WHERE temp.ID = IndividualStockAcount.ID AND temp.StockCode = IndividualStockAcount.StockCode
    )
    WHERE EXISTS (
        SELECT 1
        FROM temp
        WHERE temp.ID = IndividualStockAcount.ID AND temp.StockCode = IndividualStockAcount.StockCode
    )
"""
)
cur.execute("DROP TABLE IF EXISTS temp")
conn.commit()

## 1.2.3. Update the [UserInfo].FavCategory column
- 각 Id마다의 Category별 TotalPrice합계를 구하고, 가장 큰 Category를 FavoriteCategory로 지정

In [15]:
# Get the category with the highest sum of TotalPrice for each ID
sql = """
SELECT IndividualStockAcount.ID, StockInfo.Category, SUM(IndividualStockAcount.TotalPrice) as Total
FROM UserInfo
LEFT JOIN IndividualStockAcount ON UserInfo.ID = IndividualStockAcount.ID
LEFT JOIN StockInfo ON IndividualStockAcount.StockCode = StockInfo.StockCode
GROUP BY IndividualStockAcount.ID, StockInfo.Category
"""

df = pd.read_sql_query(sql, conn)

# Find the category with the highest sum of TotalPrice for each ID
df = df.loc[df.groupby("ID")["Total"].idxmax()]

# Update the UserInfo table with the FavCategory
for index, row in df.iterrows():
    sql = f"""
    UPDATE UserInfo
    SET FavCategory = '{row['Category']}' 
    WHERE ID = '{row['ID']}'
    """
    cur.execute(sql)

conn.commit()

In [16]:
##

In [17]:
df

,ID,Category,Total
1,jiho266,전기전자,867000.0
2,jiho267,서비스업,808000.0
3,jiho268,전기전자,649500.0
5,jiho270,전기전자,202400.0
